In [6]:
import pickle
import numpy as np
import tensorflow as tf
import keras

import os
import datetime

from keras.layers import Dense
from Chess import construct_board
from Chess.view import view_board_colour
from Chess.state import Board
from sklearn.model_selection import train_test_split

from ipywidgets import interact
import ipywidgets as widgets

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

In [7]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


### Open our set of randomly selected Board objects
The script `generate_data.py` pickles a list of board objects after a sequence of moves is run through. This leaves us with a long list of `Board` objects which we can now use!

In [9]:
with open("generated_data/all_boards.csv", 'r') as f:
    boards = f.readlines()

board_list = [construct_board(i) for i in boards]

In [10]:
view_board_colour(board_list[0])
view_board_colour(board_list[1])

Black to move - turn 3
 R  N  B  Q  K  B  N  R 
 P     P  P     P  P  P 
    P        P          
                        
          P  P          
 P                      
    P  P        P  P  P 
 R  N  B  Q  K  B  N  R 

Black to move - turn 8
 R  N     Q  K        R 
 P  P  P        P  P    
          B  P  N  P    
          P             
          P  P          
          B             
 P  P  P  N     P  P  P 
 R     B  Q  K        R 



## Data processing
Now the data needs to be formed into a reasonable type for training. This section will get all the positions which can be moved to as a list for each game, and all the positions of the pieces for each board as training data. Initally, this won't account for the piece type, which side is to move, or the different types of pieces and moves.

In [11]:
# Using set comprehension to get only the unique elements of each boards moves (the target data)
valid_move_list = [list({i for i in board.allied_moves.all_valid}) for board in board_list]

# We are targeting the correct prediction of each of these squares as "active"
len(valid_move_list[0])

19

In [12]:
# Get all the piece types and positions from the location map of each board (training data)
piece_position_list = [{position: piece.kind for position, piece in board.loc_map.items()} for board in board_list]

# This will be at most 32 pieces
len(piece_position_list[0])

32

In [13]:
# Encode the location of any piece as a 1, and all other positions as 0
feature = []
for instance in piece_position_list:
    array = np.zeros((8, 8))
    for location in instance:
        array[location.i, location.j] = 1
    feature.append(array.flatten())

In [14]:
# Simple encoding will be used at first, where any piece present is represented by a one and not present by zero.
target = []
for instance in valid_move_list:
    array = np.zeros((8, 8))
    for location in instance:
        array[location.i, location.j] = 1
    target.append(array.flatten())
    
print(target[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1.
 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [15]:
X = np.array(feature)
Y = np.array(target)

print(X.shape)
print(Y.shape)

(20, 64)
(20, 64)


The data needs to be split into training and testing portions using the `sklearn` modules implementation.

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

Beginning with a simple sequential model, which takes in a 1x64 array of features and outputs a prediction at each of the 64 squares.

In [17]:
# Inspired by the model at https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

model = keras.Sequential()
model.add(Dense(128, input_dim=64, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='sigmoid'))

Metal device set to: Apple M1


2021-12-03 12:53:45.786958: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-03 12:53:45.787873: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [18]:
model.compile(
    loss="binary_crossentropy", 
    optimizer="adam", 
    metrics=["accuracy", "binary_accuracy"]
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                16448     
Total params: 57,792
Trainable params: 57,792
Non-trainable params: 0
_________________________________________________________________


In [19]:
# From https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

2021-12-03 12:53:47.237978: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-03 12:53:47.238176: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-03 12:53:47.239280: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


In [21]:
model.fit(
    X_train, 
    Y_train, 
    epochs=100, 
    batch_size=20,
    callbacks=[tensorboard_callback]
)

Epoch 1/100
1/1 [==============================] - 0s 35ms/step - loss: 0.0161 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 2/100
1/1 [==============================] - 0s 17ms/step - loss: 0.0154 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 9ms/step - loss: 0.0149 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 8ms/step - loss: 0.0143 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 11ms/step - loss: 0.0137 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 10ms/step - loss: 0.0132 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 13ms/step - loss: 0.0127 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 9ms/step - loss: 0.0123 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 9/100

2021-12-03 12:54:06.391389: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-12-03 12:54:06.391405: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-12-03 12:54:06.405725: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-03 12:54:06.406267: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-12-03 12:54:06.406731: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20211203-125347/train/plugins/profile/2021_12_03_12_54_06
2021-12-03 12:54:06.407175: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/20211203-125347/train/plugins/profile/2021_12_03_12_54_06/Elliss-MacBook-Air.local.trace.json.gz
2021-12-03 12:54:06.407712: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/20211203-125347/train/plugins

Epoch 13/100
1/1 [==============================] - 0s 8ms/step - loss: 0.0102 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 14/100
1/1 [==============================] - 0s 8ms/step - loss: 0.0098 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 15/100
1/1 [==============================] - 0s 7ms/step - loss: 0.0095 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 16/100
1/1 [==============================] - 0s 8ms/step - loss: 0.0092 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 17/100
1/1 [==============================] - 0s 8ms/step - loss: 0.0088 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 18/100
1/1 [==============================] - 0s 8ms/step - loss: 0.0085 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 19/100
1/1 [==============================] - 0s 8ms/step - loss: 0.0083 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 20/100
1/1 [==============================] - 0s 8ms/step - loss: 0.0080 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 21

1/1 [==============================] - 0s 11ms/step - loss: 0.0021 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 79/100
1/1 [==============================] - 0s 9ms/step - loss: 0.0021 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 80/100
1/1 [==============================] - 0s 11ms/step - loss: 0.0021 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 81/100
1/1 [==============================] - 0s 10ms/step - loss: 0.0020 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 82/100
1/1 [==============================] - 0s 12ms/step - loss: 0.0020 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 83/100
1/1 [==============================] - 0s 9ms/step - loss: 0.0020 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 84/100
1/1 [==============================] - 0s 10ms/step - loss: 0.0019 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 85/100
1/1 [==============================] - 0s 9ms/step - loss: 0.0019 - accuracy: 0.0625 - binary_accuracy: 1.0000
Epoch 86/100
1/1

In [22]:
loss, accuracy, binary_accuracy = model.evaluate(X_test, Y_test)

1/1 [==============================] - 0s 277ms/step - loss: 1.0280 - accuracy: 0.0000e+00 - binary_accuracy: 0.7422


2021-12-03 12:54:08.436298: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


## Visualising the output
Now both the raw output of the model and the binarised version can be displayed interactively.

In [23]:
%tensorboard --logdir logs

In [24]:
def predict_val(val):
    prediction = model.predict(val.reshape(1, 64))
    prediction = prediction.reshape(8, 8)
    return np.where(prediction > 0.5, 1, 0)

def predict_raw(val):
    prediction = model.predict(val.reshape(1, 64))
    return prediction.reshape(8, 8)    

In [25]:
def find_diff(n):
    Y_instance = Y_test[n].reshape(8, 8)
    prediction = predict_val(X_test[n])
    
    diff = np.where(prediction == Y_instance, 1, 0)

    return diff

In [26]:
sns.set_theme(style="white")
sns.color_palette("crest", as_cmap=True)
sns.set_palette("crest")

def plot_sns(val):
    # From https://seaborn.pydata.org/examples/many_pairwise_correlations.html
    sns.heatmap(val)
    plt.show()

def plot(val):
    cmap = colors.ListedColormap(['white', 'green'])
    bounds = [-0.5, 0.5, 1.5]
    norm = colors.BoundaryNorm(bounds, cmap.N)

    fig, ax = plt.subplots()
    ax.imshow(val, cmap=cmap, norm=norm)

    # draw gridlines
    ax.grid(which='major', axis='both', linestyle='-', color='k', linewidth=2)
    ax.set_xticks(np.arange(0.5, 8.5, 1));
    ax.set_yticks(np.arange(0.5, 8.5, 1));

#     sns.heatmap(val)
    plt.show()

def plot_diff(val):
    cmap = colors.ListedColormap(['red', 'green'])
    bounds = [-0.5, 0.5, 1.5]
    norm = colors.BoundaryNorm(bounds, cmap.N)

    fig, ax = plt.subplots()
    ax.imshow(val, cmap=cmap, norm=norm)

    # draw gridlines
    ax.grid(which='major', axis='both', linestyle='-', color='k', linewidth=2)
    ax.set_xticks(np.arange(0.5, 8.5, 1));
    ax.set_yticks(np.arange(0.5, 8.5, 1));
    
#     sns.heatmap(val)
    plt.show()

In [27]:
def generate_plots(
            n, target=True, 
            prediction=True, 
            raw_prediction=True, 
            diff=True
        ):
    if target:         plot(Y_test[n].reshape(8, 8))    
    if prediction:     plot(predict_val(X_test[n]))
    if raw_prediction: plot_sns(predict_raw(X_test[n]))
    if diff:           plot_diff(find_diff(n))

In [28]:
# From https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html
interact(
    generate_plots, 
    n=widgets.IntSlider(min=0, max=199, step=5, value=0),
    target=False,
    prediction=False,
    raw_prediction=False,
    diff=True
)

interactive(children=(IntSlider(value=0, description='n', max=199, step=5), Checkbox(value=False, description=…

<function __main__.generate_plots(n, target=True, prediction=True, raw_prediction=True, diff=True)>